In [ ]:
class Block:
    
    def __init__(self, data, previous_hash, time):
        self.data = data
        self.previous_hash = previous_hash
        self.time = time
    
    def get_current_hash(self, data):
        hash_val = hashlib.sha256(bytes(data))
        return hash_val

In [3]:
Blockchain = list()

In [4]:
def create_genesis_block(data):
    hash_val = hashlib.sha256(bytes(data))
    blk = Block(data=data, previous_hash=hash_val.hexdigest(), time=datetime.timestamp(datetime.now()))
    Blockchain.append(blk)

In [5]:
def next_block(data, previous_block):
    hash_val = previous_block.get_current_hash(data)
    blk = Block(data=data, previous_hash=hash_val.hexdigest(), time=datetime.timestamp(datetime.now()))
    Blockchain.append(blk)

In [6]:
create_genesis_block(123)
for i in range(124,127):
    next_block(i, Blockchain[-1])
for blocks in Blockchain:
    print(blocks.data)
    print(blocks.previous_hash)

NameError: name 'Block' is not defined

### Socket Programming-Threads

In [54]:
import threading
import socket
import pickle

In [71]:
# client side

class Peer:
    
    def __init__(self, ip):
        self.MANAGER_IP = '192.168.43.228'
        self.MANAGER_PORT = 8097
        self.server_port = 5067
        self.client_ip = ip
        self.client_socket = socket.socket()
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0)
        self.server_thread = threading.Thread(target=self.start_server)
        self.server_thread.daemon = True
        self.server_thread.start()
        self.stop_server = False
                
    def start_server(self):
        self.server_socket.bind((self.client_ip, self.server_port))
        self.server_socket.listen(10)
        while True:
            client_peer, address = self.server_socket.accept()
            # actions
            data = client_peer.recv(4096).decode('utf-8')
            # add  
            if data.startswith('add-peer'):
                self.peers.add(data.split()[1])
            if self.stop_server:
                break
        
    def _stop_server(self):
        self.stop_server = True
        
    def _connect_to_network(self, message):
        sock = socket.socket()
        sock.connect((self.MANAGER_IP, self.MANAGER_PORT))
        sock.send(message.encode('utf-8'))
        peers = pickle.loads(sock.recv(4096))
        self.peers = peers
        
    def join_network(self):
        return self._connect_to_network(f'add {self.client_ip}')


In [72]:
client = Peer('192.168.43.152')

In [73]:
client.join_network()

In [74]:
client.peers

{'192.168.43.24'}

In [77]:
a = [375, 750, 723, 662, 647, 656, 619]

In [87]:
x = zip(a, a[1:], a[2:], a[3:], a[4:])

In [98]:
a = [375, 750, 723, 662, 647, 656, 619]
n = len(a)
ans = 1
for i in range(1,min(5, n)):
    ans = ans + 1 if a[i] < a[:i] else 0 
print(ans)

TypeError: '<' not supported between instances of 'int' and 'list'

[750, 723, 662, 647, 656, 619]